In [1]:
import httpx
import urllib.parse
from datetime import date
import pandas as pd
from functools import reduce
from scipy.stats import nbinom

In [2]:
WEATHER_API_KEY="96F785YBQ6H9WSZF4CAMDWZWM"
WEATHER_API_URL="https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"
# PLACE="San Antionio, TX"
PLACE="Lansing, MI"

In [14]:
def get_cities():
    df = pd.read_csv('data/capitals.csv')
    df['city_state'] = df.apply(lambda row: row["Capital"] + ", " + row["Abbreviation"], axis=1)
    return df['city_state'].tolist()

def get_dates():
    return [date(year,12,25) for year in range(2000,2024)]
    
def get_history(query_city):
    
    def build_query(query_city, query_date):
        return f"{WEATHER_API_URL}{urllib.parse.quote(query_city)}/{str(query_date)}/{str(query_date)}?unitGroup=metric&include=days&key={WEATHER_API_KEY}&contentType=json"
    
    def get_json(query_city, query_date):
        query = build_query(query_city, query_date)
        r = httpx.get(query)
        return r.json()
        
    dates = get_dates()
    history = []
    for date in dates:
        history.append(get_json(query_city, date))
    return history

def get_snow_from_json(history):
    snow_amount = history['days'][0]['snow']
    if snow_amount:
        return snow_amount
    else:
        return 0.0

def get_most_recent_white_christmas(history):
    def _last(year, date_snow_tuple):
        new_year = date_snow_tuple[0].year
        is_snow = date_snow_tuple[1]
        if is_snow:
            return new_year
        else:
            return year
    dates = get_dates()
    snow_amount = map(get_snow_from_json, history)
    snow_bool = map(lambda x: x > 0, snow_amount)
    combined = zip(dates, snow_bool)
    return reduce(_last,combined, None)
    
def predict_next_white_christmas(history):
    dates = get_dates()
    snow_amount = map(get_snow_from_json, history)
    snow_bool = list(map(lambda x: x > 0, snow_amount))
    probability = sum(snow_bool)/len(snow_bool)
    if probability > 0:
        ppf = nbinom.ppf(q=0.95,n=1,p=probability) # 95% confidence to the next snow (n=1) given probability of snow (history)
        return int(ppf) # number of years to wait
    else:
        return None

def rest_endpoint(PLACE):
    history = get_history(PLACE)
    last_snow = get_most_recent_white_christmas(history)
    if last_snow:
        next_snow = date.today().year + predict_next_white_christmas(history)
        return f"The last White Christmas in {PLACE} was in {last_snow}. Based on data since 2000, there will likely (with 95% confidence) be another White Christmas in {PLACE} before {next_snow}."
    else:
        return f"There has not been a White Christmas in {PLACE} since 2000. Don't hold your breath for the next one."

In [16]:
rest_endpoint("Southhold, NY")

'The last White Christmas in Southhold, NY was in 2002. Based on data since 2000, there will likely (with 95% confidence) be another White Christmas in Southhold, NY before 2094.'